<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumToyAlgorithm_CHSHGate_01_GoogleAIStudio_mkbahk_20250414.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-aer==0.15.1
%pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.1 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=7f55fe602b867a5d9b9c259c6f5fbae414d537d5b38ac1f67a88b2545c5b42d1
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 72.4 MB/s eta 0:00:00
  Cloning https://github.com/qisk

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_state_city, plot_bloch_multivector
from qiskit_textbook.tools import array_to_latex
from qiskit_aer import Aer
from IPython.display import display, Math
import numpy as np

In [7]:
# --- CHSH 게임 회로 생성 함수 ---
def create_chsh_circuit(x, y):
    """
    주어진 입력 x, y에 대한 CHSH 게임 양자 회로를 생성합니다.

    Args:
        x (int): 앨리스의 입력 (0 또는 1)
        y (int): 밥의 입력 (0 또는 1)

    Returns:
        QuantumCircuit: CHSH 게임을 위한 양자 회로
    """
    qc = QuantumCircuit(2, 2) # 큐비트 2개, 클래식 비트 2개

    # 1. 벨 상태 |Φ+> 생성 (H on q0, CX q0->q1)
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier()

    # 2. 앨리스의 측정 각도 설정 (x 값 기준)
    # x=0: Z 측정 (각도 0) -> 추가 게이트 없음
    if x == 1:
        # x=1: X 측정 (각도 pi/2) -> H 게이트 적용
        qc.h(0)
    ###if

    # 3. 밥의 측정 각도 설정 (y 값 기준)
    # Qiskit에서 특정 각도 측정은 해당 각도의 역회전을 적용 후 Z 측정
    # Ry(-theta) 적용 후 Z 측정 == theta 각도로 측정
    if y == 0:
        # y=0: 각도 pi/4 로 측정 -> Ry(-pi/4) 적용
        qc.ry(-np.pi / 4, 1)
    else: # y == 1
        # y=1: 각도 3*pi/4 로 측정 -> Ry(-3*pi/4) 적용
        qc.ry(-3 * np.pi / 4, 1)
    ###if

    qc.barrier()

    # 4. 측정 (Z 기저에서)
    # 앨리스의 결과 -> 클래식 비트 0 (a)
    # 밥의 결과 -> 클래식 비트 1 (b)
    qc.measure([0, 1], [0, 1])

    return qc
###def

In [10]:
# --- 시뮬레이터 설정 ---
simulator = Aer.get_backend("qasm_simulator")
shots = 8192 # 통계적 정확도를 위한 실행 횟수

# --- 승률 계산 ---
total_win_probability = 0
results = {}

print("CHSH 게임 시뮬레이션 시작 (Shots = {})".format(shots))
print("-" * 30)

CHSH 게임 시뮬레이션 시작 (Shots = 8192)
------------------------------


In [11]:
# 모든 입력 조합 (x, y)에 대해 실행
for x in [0, 1]:
    for y in [0, 1]:
        # 1. 해당 x, y에 대한 회로 생성
        chsh_circuit = create_chsh_circuit(x, y)
        # print(f"\n회로 (x={x}, y={y}):")
        # print(chsh_circuit) # 회로 구조 출력 (선택 사항)

        # 2. 시뮬레이션 실행
        compiled_circuit = transpile(chsh_circuit, simulator)
        job = simulator.run(compiled_circuit, shots=shots)
        result = job.result()
        counts = result.get_counts(chsh_circuit)
        results[(x, y)] = counts

        # 3. 승리 확률 계산
        win_count = 0
        target_condition = x * y # a XOR b == x*y 이어야 승리

        for outcome, count in counts.items():
            # Qiskit 결과는 'ba' 순서 (q1 측정결과, q0 측정결과)
            b = int(outcome[0]) # 밥의 결과 (q1 -> classical bit 1)
            a = int(outcome[1]) # 앨리스의 결과 (q0 -> classical bit 0)

            # 승리 조건 확인: a XOR b == x * y
            if (a ^ b) == target_condition:
                win_count += count
            ###if
        ###for

        win_probability_xy = win_count / shots
        total_win_probability += win_probability_xy

        print(f"입력 (x={x}, y={y}):")
        print(f"  측정 결과: {counts}")
        print(f"  승리 조건 (a⊕b == x*y = {target_condition}): {win_count} / {shots}")
        print(f"  승률 P(win|x={x}, y={y}): {win_probability_xy:.4f}")
        print("-" * 10)
    ###for
###for

입력 (x=0, y=0):
  측정 결과: {'01': 605, '10': 598, '00': 3486, '11': 3503}
  승리 조건 (a⊕b == x*y = 0): 6989 / 8192
  승률 P(win|x=0, y=0): 0.8531
----------
입력 (x=0, y=1):
  측정 결과: {'01': 3450, '00': 615, '10': 3460, '11': 667}
  승리 조건 (a⊕b == x*y = 0): 1282 / 8192
  승률 P(win|x=0, y=1): 0.1565
----------
입력 (x=1, y=0):
  측정 결과: {'11': 3542, '01': 591, '10': 586, '00': 3473}
  승리 조건 (a⊕b == x*y = 0): 7015 / 8192
  승률 P(win|x=1, y=0): 0.8563
----------
입력 (x=1, y=1):
  측정 결과: {'01': 594, '11': 3486, '10': 634, '00': 3478}
  승리 조건 (a⊕b == x*y = 1): 1228 / 8192
  승률 P(win|x=1, y=1): 0.1499
----------


In [12]:
# --- 최종 결과 출력 ---
average_win_probability = total_win_probability / 4
theoretical_max_win_prob = (2 + np.sqrt(2)) / 4

print("\n" + "=" * 30)
print("CHSH 게임 최종 결과")
print("=" * 30)
print(f"시뮬레이션을 통한 평균 승률: {average_win_probability:.4f}")
print(f"양자역학 이론적 최대 승률: cos^2(pi/8) ≈ {theoretical_max_win_prob:.4f}")
print(f"고전적 전략 최대 승률: 0.7500")
print("=" * 30)

# 입력 (1,1)의 결과 히스토그램 예시 (선택 사항)
# print("\n입력 (x=1, y=1) 결과 분포:")
# plot_histogram(results[(1,1)]).show()


CHSH 게임 최종 결과
시뮬레이션을 통한 평균 승률: 0.5040
양자역학 이론적 최대 승률: cos^2(pi/8) ≈ 0.8536
고전적 전략 최대 승률: 0.7500
